In [3]:
import urllib.request
import datetime
import json
import ssl

client_id = 'OfUhh9NpmXzZeWhUNB8H'
client_secret = 'jECI0GhNyV'

def getRequestUrl(url):
    # Create an SSL context to handle SSL verification
    context = ssl.create_default_context()
    context.check_hostname = False
    context.verify_mode = ssl.CERT_NONE

    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)

    try:
        # Pass the context to the urlopen function to handle SSL
        response = urllib.request.urlopen(req, context=context)
        if response.getcode() == 200:
            print("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')

    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

In [29]:
#[CODE 2]
def getNaverSearch(node, srcText, start, display):
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" %(urllib.parse.quote(srcText), start, display)
    url = base + node + parameters
    responseDecode = getRequestUrl(url) #[CODE 1]
    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)


In [30]:
#[CODE 3]
def getPostData(post, jsonResult, cnt):
    title = post['title']
    description = post['description']
    org_link = post['originallink']
    link = post['link']
    pDate = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')
    jsonResult.append({'cnt':cnt, 'title':title, 'description': description,
    'org_link':org_link, 'link': link, 'pDate':pDate})
    return

In [31]:
#[CODE 0]
def main():
    node = 'news' #크롤링할 대상
    srcText = input('검색어를 입력하세요: ')
    cnt = 0
    jsonResult = []
    jsonResponse = getNaverSearch(node, srcText, 1, 100) #[CODE 2]
    total = jsonResponse['total']
    while ((jsonResponse != None) and (jsonResponse['display'] != 0)):
        for post in jsonResponse['items']:
            cnt += 1
            getPostData(post, jsonResult, cnt) #[CODE 3]
        start = jsonResponse['start'] + jsonResponse['display']
        jsonResponse = getNaverSearch(node, srcText, start, 100) #[CODE 2]
    print('전체 검색 : %d 건' %total)
    with open('%s_naver_%s.json' % (srcText, node), 'w', encoding='utf8') as outfile:
        jsonFile = json.dumps(jsonResult, indent = 4, sort_keys = True, ensure_ascii = False)
        outfile.write(jsonFile)
    print("가져온 데이터 : %d 건" %(cnt))
    print('%s_naver_%s.json SAVED' % (srcText, node))

if __name__ == '__main__':
    main()

[2025-03-04 14:01:52.533165] Url Request Success
[2025-03-04 14:01:52.760415] Url Request Success
[2025-03-04 14:01:53.493880] Url Request Success
[2025-03-04 14:01:53.736398] Url Request Success
[2025-03-04 14:01:54.053546] Url Request Success
[2025-03-04 14:01:54.360025] Url Request Success
[2025-03-04 14:01:54.608436] Url Request Success
[2025-03-04 14:01:55.150789] Url Request Success
[2025-03-04 14:01:55.446566] Url Request Success
[2025-03-04 14:01:55.725749] Url Request Success
HTTP Error 400: Bad Request
[2025-03-04 14:01:55.823037] Error for URL : https://openapi.naver.com/v1/search/news.json?query=%EC%95%A0%ED%94%8C&start=1001&display=100
전체 검색 : 1619455 건
가져온 데이터 : 1000 건
애플_naver_news.json SAVED
